In [1]:
from MaxCutProblem import MaxCutProblem

In [2]:
problem = MaxCutProblem()

In [ ]:
import itertools
import time
import params
from QAOA import QAOArunner
from datetime import date
import pandas as pd
from solver import Solver
import ray
import numpy as np
import yagmail

with open("email_credentials.txt", "r") as f:
    email_password = f.read().strip()

@ray.remote
def parallell_runner(parameters, graph,name):
    qaoa = QAOArunner(graph, simulation=True, param_initialization=parameters[1],qaoa_variant=parameters[0], warm_start=parameters[2])
    qaoa.build_circuit()
    qaoa.run()
    solver = Solver(graph)
    bitstring, value = solver.solve()
    return {'param_initialization': parameters[1], 'qaoa_variant': parameters[0], 'warm_start' : parameters[2],
        'depth': params.depth, 'graph_size': len(graph.nodes()), 'graph_name' : name,
        'time_elapsed': qaoa.time_elapsed, 'quantum_func_evals': qaoa.fev, 'obj_func_evolution': qaoa.objective_func_vals,
        'quantum_solution':qaoa.solution, 'quantum_obj_value' : qaoa.objective_value, 
        'classic_solution' : bitstring, 'classic_value': value }


if ray.is_initialized():
    ray.shutdown()
    print('Shutting down old Ray instance.')
ray.init(num_cpus=15)

iterables = ['multiangle', params.supported_param_inits, [True,False]] 
settings = list(itertools.product(*iterables))
print(settings)

data = []

for parameters in settings:
    print('Parameters:', parameters)
    graphs = []
    names = []
    for i in range(5,10):
        graphs_i, names_i = problem.get_test_graphs(i)
        graphs.append(graphs_i) #TODO: check that this works for very small values
        names.append(names_i)

    graphs = list(itertools.chain.from_iterable(graphs))
    names = list(itertools.chain.from_iterable(names))
    futures = [parallell_runner.remote(parameters, graph, name) for graph, name in zip(graphs, names)]

    result_ids, unfinished = ray.wait(futures, timeout = 60*60*12, num_returns = len(graphs))
    for task in unfinished:
        ray.cancel(task)
    underway_df = pd.DataFrame(ray.get(result_ids))
    underway_df.to_csv(f'results_underway.csv', mode='a', header=False)
    data.extend(results)
    print(f'Done with Parameters: {parameters} at time: {time.time()}')

    
    df = pd.DataFrame(data)
    df.to_csv('data.csv', index=False)


df = pd.DataFrame(data)
df.to_csv(f'results_.csv')
yag = yagmail.SMTP("torbjorn.solstorm@gmail.com", email_password)
recipient = "torbjorn.smed@gmail.com"
subject = "Data from Python Script"
body = "Here is the data you requested."
attachment = "data.csv"

yag.send( subject=subject, contents=body, attachments=attachment)
print("Email sent successfully!")



2024-12-07 18:35:29,140	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-07 18:35:32,902	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


[('vanilla', 'uniform', True), ('vanilla', 'gaussian', True), ('multiangle', 'uniform', True), ('multiangle', 'gaussian', True), ('recursive', 'uniform', True), ('recursive', 'gaussian', True)]
Parameters: ('recursive', 'gaussian', False)
(parallell_runner pid=26296) You are running on the local simulator:  fake_brisbane


KeyboardInterrupt: 

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]


In [ ]:
underway_df = pd.DataFrame(results)
underway_df.to_csv(f'results_underway.csv', mode='a', header=False)

In [ ]:
results = ray.get(futures)
underway_df = pd.DataFrame(results)
underway_df.to_csv(f'results_underway.csv', mode='a', header=False)
data.extend(results)

RayTaskError(IndexError): [36mray::parallell_runner()[39m (pid=26928, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "C:\Users\torbjsme\AppData\Local\Temp\ipykernel_3568\852041552.py", line 13, in parallell_runner
IndexError: list index out of range

In [ ]:
results2 = []
for task in futures:
    try:
        result = ray.get(task)  # Retrieve the result
        results2.append(result)  # Store in local memory
    except Exception as e:
        print(f"Task failed with error: {e}")

Task failed with error: ray::parallell_runner() (pid=7400, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "C:\Users\torbjsme\AppData\Local\Temp\ipykernel_28600\245689880.py", line 15, in parallell_runner
  File "c:\Users\torbjsme\Master\QAOA\QAOA.py", line 224, in run
    init_params = self.get_init_params()
  File "c:\Users\torbjsme\Master\QAOA\QAOA.py", line 206, in get_init_params
    init_params = np.array([[np.random.normal(np.pi,0.2,param_cost_length), (np.random.normal(np.pi/2,0.2,param_mixer_length))] for i in range(params.depth)])
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 2) + inhomogeneous part.
Task failed with error: ray::parallell_runner() (pid=24236, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "C:\Users\torbjsme\AppData\Local\Temp\ipykernel_28600\245689880.py", line 15, in p